In [ ]:
import numpy as np
import training_data_generation as tdg
import matplotlib.pyplot as plt
import healpy as hp
import utils_intensity_map as uim
import utils_deck_generation as idg
import netcdf_read_write as nrw
import training_data_generation as tdg
import utils_healpy as uhp
import os
%matplotlib inline

savefile_type = ".png"

In [ ]:
def critical_density(wavelength_l=351.0e-9):
    epi_0 = 8.85e-12
    mass_e = 9.11e-31
    charge_e = 1.6e-19
    c_s = 3.0e8
    
    omega_l = 2.0 * np.pi * c_s / wavelength_l
    
    n_crit = epi_0 * mass_e * omega_l**2 / charge_e**2
    
    print("Critical electron density for light wavelength {:.2f}nm is {:.2e}m^-3".format(wavelength_l*1.0e9,n_crit))
    
    return n_crit


wavelength_l = 351.0e-9
n_crit = critical_density(wavelength_l)

## Diagnose plasma profiles

In [ ]:
diag_dir = "Data_240618a_N190204_003_4ns_new_weighting"#"Data_240805a_multi_N190204_003" #"Data_240618a_N190204_003_4ns_new_weighting"
iex = "N190204_003" #"240806a_N190204_003_smaller_beams" #"N190204_003"
ind_profile = 0
with_pointing_markers = True
import_flipped = False
old_format = True
display_steradians = False
sys_params = tdg.define_system_params(diag_dir)

config_location = sys_params["data_dir"] + "/" + sys_params["config_dir"] + str(iex)
run_location = config_location + "/" + sys_params["sim_dir"] + str(ind_profile)

In [ ]:
sys_params = tdg.define_system_params(diag_dir)
file_name = "ifriit_1davg_input.nc" #"multi2ifriit.nc" #"ifriit_1davg_input.nc"

input_dir = run_location + "/" + file_name
aster_profiles = nrw.read_general_netcdf(input_dir)

for key in aster_profiles.keys():
    print(key, np.shape(aster_profiles[key]))
    
#print(aster_profiles["vz"])
inc_aster = (np.abs(aster_profiles["ne"] - n_crit)).argmin()

In [ ]:
fig, ax = plt.subplots()
ax.plot([aster_profiles["xs"][inc_aster], aster_profiles["xs"][inc_aster]],[1.0e-8, 1.0],"k:")
ax.semilogy(aster_profiles["xs"], aster_profiles["ne"][0,:]/np.max(aster_profiles["ne"][0,:]))
ax.semilogy(aster_profiles["xs"], aster_profiles["te"][0,:]/np.max(aster_profiles["te"][0,:]))
ax.semilogy(aster_profiles["xs"], aster_profiles["ti"][0,:]/np.max(aster_profiles["ti"][0,:]))
ax.semilogy(aster_profiles["xs"], aster_profiles["vr"][0,:]/np.max(aster_profiles["vr"][0,:]))


## Modify aster generated Ifriit inputs

In [ ]:

output_filename = "ifriit_1davg_input_fix_species.nc"
ncells = len(aster_profiles["xs"])
nmat = 2
print("Hardcoded for CH!")
aster_profiles["atomic_index"] = np.array([1.0, 12.011])
aster_profiles["znuc"] =         np.array([1.0, 6.0])
aster_profiles["frac"] = np.zeros((nmat,1,ncells))
aster_profiles["frac"][0,:,:] = 0.5
aster_profiles["frac"][1,:,:] = 0.5

#nrw.save_general_netcdf(aster_profiles, run_location + "/" + output_filename, extra_dimension={'x': ncells, 'z':1, 'nel':nmat})

# Load profiles from multi

In [ ]:
import utils_multi as um

In [ ]:
eval_time_ns = 4.0 #0.5 #float(input()) #12.0#14.0
ind_interfaces = [0]#[300, 670]
print("!!! Hardcoded hydro evaluation time " + str(eval_time_ns) + "ns and DT-CH interface at cell:" + str(ind_interfaces)+" !!!!!!!!!!")

In [ ]:
path = "/cea/dsku/u-pelote/hal1/home/m5/barlowd/PC/duncan_work_folder/2407a_multi_build/2407a_multi/Data/Data_2408b_nif_pdd/240807a_N190204_003_reduced_energy/"
      #"/cea/dsku/u-pelote/hal1/home/m5/barlowd/PC/duncan_work_folder/2407a_multi_build/2407a_multi/Data/Data_2412a_nif_wetted_foam_rosenberg/250128a_march_design/"
      #"/cea/dsku/u-pelote/hal1/home/m5/barlowd/PC/duncan_work_folder/2407a_multi_build/2407a_multi/MULTI-IFE-2015/Data_2408c_nif_wetted_foam/240909b_pure_CH_better_resolution/"
      #240805a_N190204_003_wrong_beamspot/#240807a_N190204_003_reduced_energy/
input_filename = "multi_output.txt" #"multi_output.bin"
output_filename = "multi2ifriit.nc"

multi_data = {}
#multi_data = um.multi_read_bin(path+input_filename, multi_data)
multi_data = um.multi_read_ascii(path+input_filename)
multi_data = um.read_inputs(path+input_filename, multi_data)

itime = np.argmin(np.abs(multi_data["time"]*1.e9-eval_time_ns))
ncells = len(multi_data["x"][1,:])

list_params = ['ne', 'te', 'ti', 'vr']
list_colours = ['blue', 'orange', 'green', 'purple']
line_styles = ["-",":"]

## Convert multi to ifriit inputs

In [ ]:
multi_nc, ncells, nmat = um.multi2ifriit_inputs(multi_data, itime, ind_interfaces)
nrw.save_general_netcdf(multi_nc, path+output_filename, extra_dimension={'x': ncells, 'z':1, 'nel':nmat})
multi_nc = 0
multi_nc = nrw.read_general_netcdf(path+output_filename)

for key in multi_nc.keys():
    print(key, np.shape(multi_nc[key]))

index_heated = 100
inc_multi = (np.abs(multi_nc["ne"][0,index_heated:] - n_crit)).argmin() + index_heated
print(inc_multi)

In [ ]:
fig, ax = plt.subplots()

ax.plot(multi_data["charge_state"][itime])

In [ ]:
fig, ax = plt.subplots()

#ax.plot([aster_profiles["xs"][inc_aster], aster_profiles["xs"][inc_aster]],[1.0e-8, 1.0e1],"k-.")
ax.plot([multi_nc["xs"][inc_multi], multi_nc["xs"][inc_multi]],[1.0e-8, 1.0e1],"b:")
counter=0
for param in list_params:
    #ax.semilogy(aster_profiles["xs"], aster_profiles[param][0,:]/np.max(aster_profiles[param][0,:]), color=list_colours[counter], linestyle=line_styles[1],label=param)
    ax.semilogy(multi_nc["xs"], np.abs(multi_nc[param][0,:]/np.max(multi_nc[param][0,:])), color=list_colours[counter], linestyle=line_styles[0], label=param)
    counter+=1
ax.legend()
ax.set_xlim([0, 2000])
ax.set_title("Normalised to max value")
plt.savefig(path+"/N190204_003_comparison_aster_multi_semilog_all_params" + savefile_type, dpi=300, bbox_inches='tight')

In [ ]:
#print(multi_nc["atomic_index"])
#print(multi_nc["znuc"])
#print(multi_nc["frac"])